### Q1

### Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


### Q2

In [9]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions-hw"

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [04:09<00:00,  3.79it/s]


### Q3

In [14]:
query = 'How do I execute a command in a running docker container?'

def elastic_search(query):
    search_query = {
        "size": 5,  # Only 5 answers
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],  # Importance levels
                        "type": "best_fields"
                    }    # there is no filter 
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    scores = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        scores.append(hit['_score'])

    return result_docs, scores


results, scores = elastic_search(query)

for doc, score in zip(results, scores):
    print(f"Score: {score}, Document: {doc}")

Score: 84.050095, Document: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
Score: 75.54128, Document: {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPass

### Q4

In [16]:
query = 'How do I execute a command in a running docker container?'

def elastic_search(query):
    search_query = {
        "size": 3,  # Return 3 results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],  # Importance levels
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"   # filtering part
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source']['question'])

    return result_docs


# Get the filtered questions
questions = elastic_search(query)

# Print the 3rd question
print(f"The 3rd question returned by the search engine is: '{questions[2]}'")

The 3rd question returned by the search engine is: 'How do I copy files from a different folder into docker container’s working directory?'


### Q5

In [27]:
# query = 'How do I execute a command in a running docker container?'

def elastic_search(query):
    search_query = {
        "size": 3,  # Return 3 results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],  # Importance levels
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        question = hit['_source']['question']
        text = hit['_source']['text']
        result_docs.append((question, text))

    return result_docs


results = elastic_search(query)

context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


context = "\n\n".join([context_template.format(question=q, text=t) for q, t in results])


prompt = prompt_template.format(question=question, context=context)


print(prompt)
print(f"Length of the prompt: {len(prompt)}")


You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

### Q6

In [31]:
from groq import Groq
import os

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

query = "How do I execute a command in a running docker container?"

response = client.chat.completions.create(
    messages=[{"role": "user", "content": query}],
    model="llama3-8b-8192",
)

In [36]:
!pip install tiktoken

import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

tokens = encoding.encode(prompt)

num_tokens = len(tokens)

print(f"Number of tokens in the prompt: {num_tokens}")

Number of tokens in the prompt: 322



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Q7

In [38]:
response = client.chat.completions.create(
    messages=[
    {
    "role":"user",
    "content":prompt
    }
],
 model="llama3-8b-8192",
)

print(response.choices[0].message.content)

Based on the provided context, to execute a command in a running Docker container, you can follow these steps:

1. Find the container ID using the command: `docker ps`
2. Execute the command in the specific container using: `docker exec -it <container-id> bash`

Note that the `-it` option stands for "interactive" and "tty" mode, which allows the command to be executed as if you were directly logging into the container.
